# Shapes dataset: collect shapes

In htis folder, we are
building a dataset geospatial shapes. The actual entities and their specific types
don't matter. 
This dataset is to be a collection of geometries that include span all six standard types --
Point, LineString, Polygon, MultiPoint, MultiLineString, and MultiPolygon.
The collection of shapes
also needs exhibit various pairwise spatial relationships like adjacency, containment, 
and intersection.

All shapes will be pulled from OpenStreetMap
using the `osmnx` package. 
Since OSM is short on the Multi* type of entities, I will create them 
by combining random subsets of the other types.

A key point here is that we will not be encoding things in lon/lat space.
Instead we will focus on rectangular subsets of say 10km x 10 km. 
The mapping between lon/lat and local x/y will be done based on a local
transverse Mercator projection.




In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx
import shapely
import pyproj

import plotly
from plotly.subplots import make_subplots
from plotly.graph_objects import Scatter

from geo_encodings.vis import px_draw

## Setup

In [ ]:
# Define a lon/lat center and a radius from which to pull shapes.
# center_lat, center_lon = 43.000273, -71.088411 # Brentwood NH
# center_lat, center_lon = 43.283307, -72.576623 # Chester VT
# center_lat, center_lon = 42.631024, -70.993787 # Boxford MA
# center_lat, center_lon = 41.688562, -73.030388 # Terryville CT
# center_lat, center_lon = 42.681636, -74.920189 # Middlefield NY
center_lat, center_lon = 42.006502, -71.808369 # Thompson CT

extent = 30000.0 # meters

# Props to ChatGPT for this:
colors = [
    "#1f77b4",  # Blue (Good for water features)
    "#ff7f0e",  # Orange (Great for roads or paths)
    "#2ca02c",  # Green (Natural features like parks)
    "#d62728",  # Red (Important landmarks or warnings)
    "#9467bd",  # Purple (Alternative highlight color)
    "#8c564b",  # Brown (Earthy tones for terrain)
    "#e377c2",  # Pink (Soft accent, good for points of interest)
    "#7f7f7f",  # Gray (Neutral elements)
    "#bcbd22",  # Yellow-Green (Highlighting important areas)
    "#17becf"   # Cyan (Water-adjacent features or paths)
]


## Geo transform setup

In [ ]:
# Define a local map projection.
from pyproj import CRS, Transformer

# Define the Local Transverse Mercator (LTM) CRS
offset = (extent / 2)
proj_def = f"""
+proj=tmerc +lat_0={center_lat} +lon_0={center_lon} 
+k=1.0 +x_0={offset} +y_0={offset} +datum=WGS84 +units=m +no_defs
"""
print(proj_def)
ltm_crs = CRS.from_proj4(proj_def)

# Define WGS84 (Latitude/Longitude) CRS
wgs84_crs = CRS.from_epsg(4326)

# Create transformer for converting from lat/lon (WGS84) to LTM
proj_forward = Transformer.from_crs(wgs84_crs, ltm_crs, always_xy=True).transform
proj_inverse = Transformer.from_crs(ltm_crs, wgs84_crs, always_xy=True).transform

# Test conversion/
lon, lat = center_lon, center_lat
x, y = proj_forward(lon, lat)
print(x, y)

lon, lat = proj_inverse(x, y)
print(lon, lat)

In [ ]:
# Define a polygon defining the bounding box for our area of interest.
xx = [0, extent, extent, 0, 0]
yy = [0, 0, extent, extent, 0]
aoi = shapely.geometry.Polygon(list(zip(xx, yy)))
print(aoi)

In [ ]:
# Get the bounds to be used for querying from OSM.
lon0, lat0 = proj_inverse(0, 0)
lon1, lat1 = proj_inverse(extent, extent)
query_bounds = [lon0, lat0, lon1, lat1]
print(query_bounds)

## Base polygons
Define a bunch of "base polygons" consisting of the town boundaries within the AOI.
This will approximately tile the area, and will share borders. So treating these as a separate
class from other polygons (below) assures that we will have a reasonable number of cases of
polygon containment and polygon borderings. 

In [ ]:
# Get named towns.
tags = {
    'boundary': 'administrative',
    'admin_level': '8'
}
gdf = osmnx.features.features_from_bbox(query_bounds, tags=tags).reset_index()
iok = gdf['admin_level'] == '8'
iok = np.logical_and(iok, np.array([type(z) for z in gdf['name'].values]) == str)
towns = gdf[iok][['id', 'geometry', 'name']]
print('%d records' % len(towns))

In [ ]:
# Only keep the portions that fall within our AOI, and that are a minimum fraction of their original.
base_polygons = []
for rec in towns.itertuples():
    g1 = shapely.ops.transform(proj_forward, rec.geometry)
    g2 = g1.intersection(aoi)
    if g2.geom_type != 'Polygon':
        continue
    if g2.area / g1.area > 0.1:
        base_polygons.append(g2)
print('%d shapes retained' % len(base_polygons))

In [ ]:
fig = make_subplots(1, 1)
for k, shape in enumerate(base_polygons):
    px_draw(shape, fig, name='shape %d' % k, color=colors[k % len(colors)])
fig['layout']['width'] = 500
fig['layout']['height'] = 500
fig

## Polygons and MultiPolygons
Define a set of polygons consisting of local OSM "landuse" instances.
These are typically smaller than the town polygons, and are generally disjoint. 
We pick all examples of a few different types of landuse categories. Then to get 
a set of MultiPolygon objects, we create a few random groupings.

In [ ]:
# Get landuse polygons.
tags = { 'landuse': ['residential', 'commercial', 'industrial', 'retail', 'farmland']}
gdf = osmnx.features.features_from_bbox(query_bounds, tags=tags).reset_index()
names = [
    z['name'] if type(z['name']) == str else '_%s_' % z['landuse']
    for z in gdf.to_dict('records')
]
gdf['name'] = names
columns = ['id', 'landuse', 'name', 'geometry']
landuse = gdf[columns]
print('%d records' % len(landuse))

In [ ]:
# Only keep the portions that fall within our AOI, and that are a minimum fraction of their original size.
polygons = []
for rec in landuse.itertuples():
    g0 = rec.geometry
    g1 = shapely.ops.transform(proj_forward, g0)
    g2 = g1.intersection(aoi)
    if g2.geom_type != 'Polygon': # It can happen. 
        continue
    if g2.area / g1.area > 0.1 and g2.area > 10000.0:
        polygons.append(g2)
print('%d shapes retained' % len(polygons))

In [ ]:
# # Create a few multipolygon shapes.
# n_polygons = len(polygons)
# n_multipolygons = 200
# multipolygons = []
# for i in range(n_multipolygons):
#     sample_count = np.random.randint(8) + 2
#     sample_indices = np.random.choice(n_polygons, sample_count, replace=False)
#     mp = shapely.MultiPolygon([polygons[i] for i in sample_indices])
#     multipolygons.append(mp)


## Linestrings
Use major roads and waterways.

In [ ]:
# Roads
tags = { 'highway': ['motorway', 'trunk', 'primary', 'secondary']}
roads = osmnx.features.features_from_bbox(query_bounds, tags=tags).reset_index()
print('%d records' % len(roads))

In [ ]:
# Waterways
tags = {"waterway": ["river", "stream", "canal"]}
waterways = osmnx.features.features_from_bbox(query_bounds, tags=tags).reset_index()
print('%d records' % len(waterways))

In [ ]:
# Reproject and filter.
linestrings = []
for source in [roads, waterways]:
    for rec in source.itertuples():
        g0 = rec.geometry
        g1 = shapely.ops.transform(proj_forward, g0)
        g2 = g1.intersection(aoi)
        if g2.geom_type != 'LineString':
            continue
        if g2.length / g1.length > 0.5 and g2.length > 500:
            linestrings.append(g2)
print('%d shapes' % len(linestrings))

In [ ]:
# # Create a few multilinestrings shapes.
# n_linestrings = len(linestrings)
# n_multilinestrings = 200
# multilinestrings = []
# for i in range(n_multilinestrings):
#     sample_count = np.random.randint(8) + 2
#     sample_indices = np.random.choice(n_linestrings, sample_count, replace=False)
#     mp = shapely.MultiLineString([linestrings[i] for i in sample_indices])
#     multilinestrings.append(mp)


## Points

In [ ]:
tags = {"amenity": True}
amenities = osmnx.features.features_from_bbox(query_bounds, tags=tags).reset_index()
print('%d records' % len(amenities))

In [ ]:
# Reproject and filter.
# Some of these show up as polygons; just use their centroids.
points = []
for source in [amenities]:
    for rec in source.itertuples():
        if rec.amenity in ['parking', 'parking_space']: # There are just too many of these.
            continue
        g0 = rec.geometry
        if g0.geom_type == 'Point':
            g1 = g0
        elif g0.geom_type == 'Polygon':
            g1 = g0.centroid
        else:
            continue
        g2 = shapely.ops.transform(proj_forward, g1)
        if g2.within(aoi):
            points.append(g2)
print('%d shapes' % len(points))

In [ ]:
# # Create a few MultiPoint shapes.
# n_points = len(points)
# n_multipoints = 200
# multipoints = []
# for i in range(n_multipoints):
#     sample_count = np.random.randint(8) + 2
#     sample_indices = np.random.choice(n_points, sample_count, replace=False)
#     mp = shapely.MultiPoint([points[i] for i in sample_indices])
#     multipoints.append(mp)


## Put them all together

In [ ]:
shape_list = base_polygons + polygons + linestrings  + points 
types = [g.geom_type for g in shape_list]
shapes = gpd.GeoDataFrame({'type': types, 'geom': shape_list})
shapes['type'].value_counts()
print('%d shapes' % len(shapes))
print(shapes['type'].value_counts())

shape_list = base_polygons 
types = [g.geom_type for g in shape_list]
tiled_polygons = gpd.GeoDataFrame({'type': types, 'geom': shape_list})
print('\n%d tiled shapes' % len(tiled_polygons))
print(tiled_polygons['type'].value_counts())


## A thing to generate shape pairs 
That have certain types of relationshiops

In [ ]:
from generators import Generator

In [ ]:
aoi_width = 100
aoi_height = 100
relation = 'point-on-linestring'
relation = 'point-in-polygon'
relation = 'linestring-intersects-linestring'
relation = 'linestring-intersects-polygon'
relation = 'polygon-intersects-polygon'
relation = 'polygon-borders-polygon'

fodder = tiled_polygons if relation == 'polygon-borders-polygon' else shapes
generator = Generator(fodder, bounds=[0, 0, aoi_width, aoi_height], scale=25)


ncases = 4

fig = make_subplots(2, ncases)

for i in range(ncases):
    a, b = generator.generate(relation, True, max_attempts=100)
    px_draw(a, fig, irow=1, icol=i+1, name=a.geom_type, color='red')
    px_draw(b, fig, irow=1, icol=i+1, name=b.geom_type, color='blue')

for i in range(ncases):
    a, b = generator.generate(relation, False)
    px_draw(a, fig, irow=2, icol=i+1, name=a.geom_type, color='red')
    px_draw(b, fig, irow=2, icol=i+1, name=b.geom_type, color='blue')

fig['layout']['title'] = relation
fig['layout']['width'] = 1000
fig['layout']['height'] = 500

for i in range(8):
    fig['layout']['xaxis%d' % (i+1)]['range'] = [0, aoi_width]
    fig['layout']['yaxis%d' % (i+1)]['range'] = [0, aoi_height]
fig
# fig.print_grid()

In [ ]:
cases = []

#
# Most cases
#

generator = Generator(shapes, bounds=[0, 0, 100, 100], scale=25)

relations = [
    'point-on-linestring',  
    'point-in-polygon', 
    'linestring-intersects-linestring',
    'linestring-intersects-polygon', 
    'polygon-intersects-polygon',
]

for relation in relations:
    print(relation)
    for i in range(100):
        ma = 100 if relation == 'point-in-polygon' else 20
        aa, bb = generator.generate(relation, True, max_attempts=ma)
        if aa is not None and bb is not None:
            cases.append({
                'relation': relation,
                'sense': True,
                'shape_a': shapely.set_precision(aa, 0.001), 
                'shape_b': shapely.set_precision(bb, 0.001),
            })
    for i in range(200):
        aa, bb = generator.generate(relation, False)
        if aa is not None and bb is not None:
            cases.append({
                'relation': relation,
                'sense': False,
                'shape_a': shapely.set_precision(aa, 0.001), 
                'shape_b': shapely.set_precision(g2, 0.001),
            })

#
# polygon border case
# 

generator = Generator(tiled_polygons, bounds=[0, 0, 100, 100], scale=25)

relations = [
    'polygon-borders-polygon',
]

for relation in relations:
    print(relation)
    for i in range(100):
        aa, bb = generator.generate(relation, True)
        if aa is not None and bb is not None:
            cases.append({
                'relation': relation,
                'sense': True,
                'shape_a': shapely.set_precision(aa, 0.001), 
                'shape_b': shapely.set_precision(bb, 0.001),
            })
    for i in range(200):
        aa, bb = generator.generate(relation, True)
        if aa is not None and bb is not None:
            cases.append({
                'relation': relation,
                'sense': False,
                'shape_a': shapely.set_precision(aa, 0.001), 
                'shape_b': shapely.set_precision(bb, 0.001),
            })

print(len(cases))


In [ ]:
import geopandas
a = geopandas.GeoDataFrame(cases)
a.drop(columns=['shape_a', 'shape_b']).value_counts().sort_index()

In [ ]:
# Save it
import pygeohash
tag = pygeohash.encode(center_lat, center_lon, 8)
fname = 'relations-%s.geojson' % tag
out = geopandas.GeoDataFrame(cases)
out = geopandas.GeoDataFrame(out.drop_duplicates())
out.to_file(fname, driver='GeoJSON')
print('%s' % fname)